In [ ]:
! pip install -U spacy
! pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.0.0/en_core_web_lg-3.0.0.tar.gz

In [ ]:
import os
import pprint
import string
import re
import pandas as pd
import numpy as np
import plotly.express as px

import nltk
from nltk.text import Text
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.collocations import BigramCollocationFinder,BigramAssocMeasures
import spacy
from spacy import displacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics.pairwise import cosine_similarity

nltk.download('all')
pp = pprint.PrettyPrinter(indent=4, compact=True)
nlp = spacy.load("en_core_web_lg")

english_stops = set(stopwords.words('english'))
english_stops.update(string.punctuation)

In [ ]:
# Activacndo Google Drive para guardar la información
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
tedx_df = pd.read_csv('/content/gdrive/My Drive/Facultad/Austral/Maestria 1/13 - Text mining/00 - TPs/Web Scraping/TED_Talk.csv')
tedx_df.head()

In [ ]:
def remove_stop_words(text):
  """
    Remueve stop words en inglés

    Attributes
    ----------
    text: list
      lista de palabras (tokens) a filtrar

    Returns
    -------
    list
      lista de palabras sin los stop words
  """
  return [token for token in text if token.lower() not in english_stops]

In [ ]:
def lematize_words(text):
  """
    Lematización de palabras - aplica lematización de palabras sobre un set de tokens

    Attributes
    ----------
    text: list
      lista de palabras (tokens) sobre los cuales se aplicará la lematización
    
    Returns
    -------
    list
      lista con todas las lematizaciones de las palabras
  """
  doc = nlp(" ".join(text))
  return [token.lemma_ for token in doc]

In [ ]:
def stem_words(text):
  """
    Stemming de palabras - aplica stemming de palabras sobre un set de tokens

    Attributes
    ----------
    text: list
      lista de palabras (tokens) sobre las que se aplicará stemming
    
    Returns
    -------
    list
      lista con los stems de las palabras
  """
  stemmer = SnowballStemmer('english')
  return [stemmer.stem(word) for word in text]

In [ ]:
def remove_meaningless_words(text):
  """
    Remueve palabras sin significado

    Attributes
    ----------
    text: list
      lista de palabras (tokens) a filtrar
    
    Returns
    -------
    list
      lista de palabras filtrada en base a expresiones regulares
  """
  patterns = [r"(^={1,}=$)", r'\u200b']
  tokens = text
  for pattern in patterns:
    regexp = re.compile(pattern)
    tokens = [token for token in tokens if not regexp.search(token)]
  return tokens

In [ ]:
def tokenize(text, mode='word'): 
  """
    Tokenización de documento - tokeniza un documento por palabra o por oración

    Attributes
    ----------
    text: str
      documento a tokenizar
    mode: str, optional
      método de tokenización (default: 'word' (por palabra))
    
    Returns
    -------
    list
      lista de tokens 
    
    Raises
    ------
      Exception
        si el mode no es 'word' o 'sentence'
  """
  if mode == 'word':
    return word_tokenize(text, language='english')
  elif mode == 'sentence':
    return sent_tokenize(text, language='english')
  else:
    raise Exception('metodo de tokenizacion no encontrado')

In [ ]:
def similarity_btw_docs(matrix):
  """
    Similitud entre documentos - calcula la similitud entre documentos utilizando Similitud del Coseno

    Attributes
    ----------
    matrix: scipy matrix
      Matriz dispersa para calcular la similaridad

    Returns
    -------
    pd.DataFrame
      retorna un dataframe con el grado de similaridad entre documentos (de 0 a 1)
  """
  matrix_simil = cosine_similarity(matrix)
  return pd.DataFrame(matrix_simil)

In [ ]:
def pre_procesamiento_texto(text):
  """
    Pre-procesamiento y obtención de las 20 palabras más significativas

    Attributes
    ----------
    text: str
      documento a analizar

    Returns
    -------
    pd.DataFrame
      retorna un dataframe con las 20 palabras que más se repiten y su frecuencia
  """
  tokenized = tokenize(text)
  without_stops = remove_stop_words(tokenized)
  meaningfull_tokens = remove_meaningless_words(without_stops)
  lematized_words = lematize_words(meaningfull_tokens)
  # stemmed_words = stem_words(lematized_words)
  return lematized_words

In [ ]:
tedx_df = tedx_df.assign(talks__tags=tedx_df['talks__tags'].str.split(',')).explode('talks__tags')
tedx_df['tag'] = tedx_df['talks__tags'].str.replace("[", "").str.replace("'", "").str.replace("]", "")
del tedx_df['talks__tags']

In [ ]:
tedx_df = tedx_df[tedx_df['transcript'] != np.nan]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tedx_df.transcript, tedx_df.tag, test_size=.30)
X_train.head()

In [ ]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=pre_procesamiento_texto)
X_train_transform = tfidf_vectorizer.fit_transform(X_train.values.astype('U'))
X_test_transform = tfidf_vectorizer.transform(X_test.values.astype('U'))

In [ ]:
similarity_btw_docs(X_train_transform)